# 07: RDD Joins

> **Note:** This example is obsolete. Use [Dataset](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset) (SQL) joins instead. They are far more flexible and performant!

Joins are a familiar concept in databases and Spark supports them, too. Joins at very large scale can be quite expensive, although a number of optimizations have been developed, some of which require programmer intervention to use. We won't discuss the details here, but it's worth reading how joins are implemented in various *Big Data* systems, such as [this discussion for Hive joins](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Joins#LanguageManualJoins-JoinOptimization) and the **Joins** section of [Hadoop: The Definitive Guide](http://shop.oreilly.com/product/0636920021773.do).

Here, we will join the KJV Bible data with a small "table" that maps the book abbreviations to the full names, e.g., `Gen` to `Genesis`.

See the corresponding Spark job [Joins7.scala](https://github.com/deanwampler/spark-scala-tutorial/blob/master/src/main/scala/sparktutorial/Joins7.scala).

In [1]:
val in = "../data/kjvdat.txt"                       // '|' separated
val abbrevToNames = "../data/abbrevs-to-names.tsv"  // tab separated
val out = "output/kjv-joins"

in = ../data/kjvdat.txt
abbrevToNames = ../data/abbrevs-to-names.tsv
out = output/kjv-joins


output/kjv-joins

This time, we won't use the `toText` method we defined before, we'll just split on `|` as before, but return a new record, `(book, (chapter#, verse#, verse))`.

In [2]:
val input = sc.textFile(in)
  .map { line =>
    val ary = line.split("\\s*\\|\\s*")
    (ary(0), (ary(1), ary(2), ary(3)))
  }

input = MapPartitionsRDD[2] at map at <console>:30


MapPartitionsRDD[2] at map at <console>:30

Now load the abbreviations, splitting each line on whitespace (for the tab), but only on the first whitespace, since some book names have more than one word.

In [3]:
val abbrevs = sc.textFile(abbrevToNames)
  .map { line =>
    val ary = line.split("\\s+", 2)
    (ary(0), ary(1).trim)  // I've noticed trailing whitespace... so trim it
  }

abbrevs = MapPartitionsRDD[5] at map at <console>:30


MapPartitionsRDD[5] at map at <console>:30

Now, for both datasets, the key is the first element, the Bible book abbreviation, so the RDD join is simple.

In [4]:
val verses = input.join(abbrevs)

verses = MapPartitionsRDD[8] at join at <console>:34


MapPartitionsRDD[8] at join at <console>:34

Did we match every line?

In [5]:
if (input.count != verses.count) {
  println(s"input count, ${input.count}, doesn't match output count, ${verses.count}")
} else {
  println("All records were matched!")
}

All records were matched!


Let's look at a few records to see what the join produced.

In [6]:
verses.take(5).foreach(println)

(Ti1,((1,1,Paul, an apostle of Jesus Christ by the commandment of God our Saviour, and Lord Jesus Christ, which is our hope;~),1 Timothy))
(Ti1,((1,2,Unto Timothy, my own son in the faith: Grace, mercy, and peace, from God our Father and Jesus Christ our Lord.~),1 Timothy))
(Ti1,((1,3,As I besought thee to abide still at Ephesus, when I went into Macedonia, that thou mightest charge some that they teach no other doctrine,~),1 Timothy))
(Ti1,((1,4,Neither give heed to fables and endless genealogies, which minister questions, rather than godly edifying which is in faith: so do.~),1 Timothy))
(Ti1,((1,5,Now the end of the commandment is charity out of a pure heart, and of a good conscience, and of faith unfeigned:~),1 Timothy))


That's `(abbrev, ((chapter#, verse#, verse), full_name))`. Let's project out new records like the original, with the full name replacing the abbreviation. Note the use of "deep" pattern matching.

In [7]:
val verses2 = verses.map {
  // Drop the key - the abbreviated book name
  case (_, ((chapter, verse, text), fullBookName)) => (fullBookName, chapter, verse, text)
}

verses2 = MapPartitionsRDD[9] at map at <console>:36


MapPartitionsRDD[9] at map at <console>:36

In [8]:
verses2.take(5).foreach(println)

(1 Timothy,1,1,Paul, an apostle of Jesus Christ by the commandment of God our Saviour, and Lord Jesus Christ, which is our hope;~)
(1 Timothy,1,2,Unto Timothy, my own son in the faith: Grace, mercy, and peace, from God our Father and Jesus Christ our Lord.~)
(1 Timothy,1,3,As I besought thee to abide still at Ephesus, when I went into Macedonia, that thou mightest charge some that they teach no other doctrine,~)
(1 Timothy,1,4,Neither give heed to fables and endless genealogies, which minister questions, rather than godly edifying which is in faith: so do.~)
(1 Timothy,1,5,Now the end of the commandment is charity out of a pure heart, and of a good conscience, and of faith unfeigned:~)


Save the output.

In [10]:
verses2.saveAsTextFile(out)

lastException: Throwable = null


Are the books sorted properly? If not, any idea why not??

## Recap

NGrams are not only useful, they can be fun to play with...

## Exercises

### Exercise 1: Try different sacred texts

Does the Aprocrypha `apodat.txt` also work with the abbreviations file?

### Exercise 2: Try Outer Joins

See [PairRDDFunctions](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions), which is what we actually used when calling `join`. (An implicit conversion is used...)

### Exercise 3: Preserve the original order

The output does _not_ preserve the original order of the verses! This is a consequence of how joins are implemented (using "co-groups").

Fix the ordering. Here is one approach; compute (either here or in advance and stored in a file) a map from book names (or abbreviations) to an index, e.g., `Gen -> 1`, `Exo -> 2`, etc. Use this to construct a sort key containing the book index, chapter number, and verse number. Note that the chapter number and verse number will be strings when extracted from the file, so you must convert them to integers using `x.toInt`. Finally, project out the full book name, chapter, verse, and text.